In [ ]:
# Google Drive können wir folgendermaßen mounten:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
!pip install opensimplex

In [2]:
%cd /content/
!git clone https://github.com/pbaylies/stylegan2-ada
%cd /content/stylegan2-ada
!mkdir out

/content
Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 361, done.
remote: Total 361 (delta 0), reused 0 (delta 0), pack-reused 361
Receiving objects: 100% (361/361), 61.57 MiB | 23.22 MiB/s, done.
Resolving deltas: 100% (196/196), done.
/content/stylegan2-ada


In [3]:
%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


In [ ]:
# FFHQ
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl

--2022-02-17 20:16:18--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 108.159.227.118, 108.159.227.82, 108.159.227.48, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|108.159.227.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381646055 (364M) [binary/octet-stream]
Saving to: ‘ffhq.pkl’

ffhq.pkl            100%[===================>] 363.97M  20.0MB/s    in 19s     

2022-02-17 20:16:38 (18.8 MB/s) - ‘ffhq.pkl’ saved [381646055/381646055]



In [ ]:
!wget https://archive.org/download/wikiart-stylegan2-conditional-model/WikiArt_Uncond2.pkl

In [ ]:
!wget https://archive.org/download/wikiart-stylegan2-conditional-model/WikiArt5.pkl

In [ ]:
# Metfaces
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl

--2022-02-17 20:39:33--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 108.159.227.118, 108.159.227.82, 108.159.227.98, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|108.159.227.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381646596 (364M) [binary/octet-stream]
Saving to: ‘metfaces.pkl’

metfaces.pkl        100%[===================>] 363.97M   123MB/s    in 3.0s    

2022-02-17 20:39:37 (123 MB/s) - ‘metfaces.pkl’ saved [381646596/381646596]



In [ ]:
# Import Libs
import tensorflow as tf
import argparse
import sys
import os
import subprocess
import pickle
import re
from projector import Projector
import tqdm
import imageio

import scipy
import numpy as np
import PIL.Image

import dnnlib
import dnnlib.tflib as tflib

os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
import cv2
import moviepy.editor
from opensimplex import OpenSimplex

import warnings # mostly numpy warnings for me
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2605056/45929032 bytes (5.7%)5988352/45929032 bytes (13.0%)9568256/45929032 bytes (20.8%)12640256/45929032 bytes (27.5%)16039936/45929032 bytes (34.9%)19554304/45929032 bytes (42.6%)23093248/45929032 bytes (50.3%)26664960/45929032 bytes (58.1%)30236672/45929032 bytes (65.8%)33636352/45929032 bytes (73.2%)37232640/45929032 bytes (81.1%)40394752/45929032 bytes (88.0%)

In [ ]:
# Functions

def create_image_grid(images, grid_size=None):
    '''
    Args:
        images (np.array): images to place on the grid
        grid_size (tuple(int, int)): size of grid (grid_w, grid_h)
    Returns:
        grid (np.array): image grid of size grid_size
    '''
    # Some sanity check:
    assert images.ndim == 3 or images.ndim == 4
    num, img_h, img_w = images.shape[0], images.shape[1], images.shape[2]
    if grid_size is not None:
        grid_w, grid_h = tuple(grid_size)
    else:
        grid_w = max(int(np.ceil(np.sqrt(num))), 1)
        grid_h = max((num - 1) // grid_w + 1, 1)
    # Get the grid
    grid = np.zeros(
        [grid_h * img_h, grid_w * img_w] + list(images.shape[-1:]), dtype=images.dtype
    )
    for idx in range(num):
        x = (idx % grid_w) * img_w
        y = (idx // grid_w) * img_h
        grid[y : y + img_h, x : x + img_w, ...] = images[idx]
    return grid

def render_from_zs(Gs, zs, w_space=False, grid=False, class_idx=None, prefix=''):
    images = []
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    label = np.zeros([1] + Gs.input_shapes[1][1:])
    if class_idx is not None:
        label[:, class_idx] = 1
    for i_z, z in enumerate(zs):
        print('Generating image for latent code (%d/%d) ...' % (i_z, len(zs)))
        if w_space:
            image = Gs.components.synthesis.run(z, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
        else:
            image = Gs.run(z, label, **Gs_kwargs) # [minibatch, height, width, channel]
        images.append(image[0])
        PIL.Image.fromarray(image[0], 'RGB').save(f'{outdir}/z{i_z:04d}.png')

    # If user wants to save a grid of the generated images
    if grid:
        print('Generating image grid...')
        PIL.Image.fromarray(create_image_grid(np.array(images)), 'RGB').save(f'{outdir}/grid{prefix}.png')

def interpolate(za, zb, n_steps):
    zs = []
    for i in range(n_steps):
        a = i/(n_steps-1)
        z = (1-a)*za + a*zb
        zs.append(z)
    return zs

def lerp_video(Gs,                         # Path to pretrained model pkl file
               za,
               zb,
               w_space=False,
               truncation_psi=1.0,         # Truncation trick
               slowdown=1,                 # Slowdown of the video (power of 2)
               duration_sec=30.0,          # Duration of video in seconds
               smoothing_sec=3.0,
               mp4_fps=30,
               mp4_codec="libx264",
               mp4_bitrate="16M"):
    # Sanity check regarding slowdown
    message = 'slowdown must be a power of 2 (1, 2, 4, 8, ...) and greater than 0!'
    assert slowdown & (slowdown - 1) == 0 and slowdown > 0, message
    # Total duration of video and number of frames to generate
    num_frames = int(np.rint(duration_sec * mp4_fps))
    total_duration = duration_sec * slowdown

    print("Generating latent vectors...")
    all_latents = interpolate(za, zb, n_steps=num_frames)
    print(np.array(all_latents).shape)
    make_grid = False
    if np.array(all_latents).shape[1] > 1:
        make_grid = True

    #all_latents = scipy.ndimage.gaussian_filter(
    #    all_latents,
    #    [smoothing_sec * mp4_fps] + [0] * len(Gs.input_shape),
    #    mode="wrap"
    #)

    #all_latents /= np.sqrt(np.mean(np.square(all_latents)))
    # Name of the final mp4 video
    mp4 = f"lerp-{slowdown}xslowdown.mp4"

    # Aux function to slowdown the video by 2x
    def double_slowdown(latents, duration_sec, num_frames):
        # Make an empty latent vector with double the amount of frames
        z = np.empty(np.multiply(latents.shape, [2, 1, 1]), dtype=np.float32)
        # Populate it
        for i in range(len(latents)):
            z[2*i] = latents[i]
        # Interpolate in the odd frames
        for i in range(1, len(z), 2):
            # For the last frame, we loop to the first one
            if i == len(z) - 1:
                z[i] = (z[0] + z[i-1]) / 2
            else:
                z[i] = (z[i-1] + z[i+1]) / 2
        # We also need to double the duration_sec and num_frames
        duration_sec *= 2
        num_frames *= 2
        # Return the new latents, and the two previous quantities
        return z, duration_sec, num_frames

    while slowdown > 1:
        all_latents, duration_sec, num_frames = double_slowdown(all_latents, duration_sec, num_frames)
        slowdown //= 2

    # Define the kwargs for the Generator:
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    label = np.zeros([1] + Gs.input_shapes[1][1:])

    # Aux function: Frame generation func for moviepy.
    def make_frame(t):
        frame_idx = int(np.clip(np.round(t * mp4_fps), 0, num_frames - 1))
        latents = all_latents[frame_idx]
        # Get the images (with labels = None)
        if w_space:
            images = Gs.components.synthesis.run(latents, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
        else:
            images = Gs.run(latents, label, **Gs_kwargs)
        # Generate the grid for this timestamp:
        if make_grid:
            grid = create_image_grid(images)
            ## grayscale => RGB
            if grid.shape[2] == 1:
                grid = grid.repeat(3, 2)
            return grid
        return images[0]

    # Generate video using make_frame:
    print(f'Generating interpolation video of length: {total_duration} seconds...')
    videoclip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
    videoclip.write_videofile(os.path.join(outdir, mp4),
                              fps=mp4_fps,
                              codec=mp4_codec,
                              bitrate=mp4_bitrate)

def project_image_into_wspace(Gs, name, save_video=False):
    # Load target image.
    imgpath = name+'.jpg'
    print(imgpath)
    target_pil = PIL.Image.open(imgpath)
    w, h = target_pil.size
    s = min(w, h)
    target_pil = target_pil.crop(((w - s) // 2, (h - s) // 2, (w + s) // 2, (h + s) // 2))
    target_pil= target_pil.convert('RGB')
    target_pil = target_pil.resize((Gs.output_shape[3], Gs.output_shape[2]), PIL.Image.ANTIALIAS)
    target_uint8 = np.array(target_pil, dtype=np.uint8)
    target_float = target_uint8.astype(np.float32).transpose([2, 0, 1]) * (2 / 255) - 1

    # Initialize projector.
    proj = Projector()
    proj.set_network(Gs)
    proj.start([target_float])

    # Videooption
    writer = None
    if save_video:
        writer = imageio.get_writer(f'{outdir}/{name}_proj.mp4', mode='I', fps=60, codec='libx264', bitrate='16M')

    # Run projector.
    with tqdm.trange(proj.num_steps) as t:
        for step in t:
            assert step == proj.cur_step
            if writer is not None:
                writer.append_data(np.concatenate([target_uint8, proj.images_uint8[0]], axis=1))
            dist, loss = proj.step()
            t.set_postfix(dist=f'{dist[0]:.4f}', loss=f'{loss:.2f}')

    # Save results.
    PIL.Image.fromarray(proj.images_uint8[0], 'RGB').save(f'{outdir}/{name}_proj.png')
    np.savez(f'{outdir}/{name}_dlatents.npz', dlatents=proj.dlatents)
    return proj.dlatents

In [ ]:
# Pfad zu Model spezifizieren
modelpfad = "metfaces.pkl"

# Model laden
tflib.init_tf()
with dnnlib.util.open_url(modelpfad) as fp:
    _G, _D, Gs = pickle.load(fp)

In [ ]:
# output ordner für renders erstellen
outdir = "out"
os.makedirs(outdir, exist_ok=True)

3.0 "Der Latentspace ist einigermaßen kontinuierlich."

In [ ]:
# 12 Zufällige Bilder generieren.
zs = np.random.normal(0,1,(12,1,512))
render_from_zs(Gs, zs, grid=True)

Generating image for latent code (0/10) ...
Generating image for latent code (1/10) ...
Generating image for latent code (2/10) ...
Generating image for latent code (3/10) ...
Generating image for latent code (4/10) ...
Generating image for latent code (5/10) ...
Generating image for latent code (6/10) ...
Generating image for latent code (7/10) ...
Generating image for latent code (8/10) ...
Generating image for latent code (9/10) ...
Generating image grid...


In [ ]:
# Zwischen 2 zufälligen Bildern interpolieren.
za = np.random.normal(0,1,(1,512))
zb = np.random.normal(0,1,(1,512))
zs = interpolate(za, zb, 12)
render_from_zs(Gs, zs, w_space=False, grid=True, class_idx=None)

In [ ]:
# Video-Interpolation zwischen 2 zufälligen Bildern
za = np.random.normal(0,1,(1,512))
zb = np.random.normal(0,1,(1,512))
zs = interpolate(za, zb, 12)
render_from_zs(Gs, zs, grid=True, class_idx=None)
lerp_video(Gs,
          za,
          zb,
          w_space=False
          truncation_psi=1.0,         # Truncation trick
          slowdown=1,                 # Slowdown of the video (power of 2)
          duration_sec=3.0,          # Duration of video in seconds
          smoothing_sec=0.4,
          mp4_fps=30,
          mp4_codec="libx264",
          mp4_bitrate="16M")

Generating image for latent code (0/12) ...
Generating image for latent code (1/12) ...
Generating image for latent code (2/12) ...
Generating image for latent code (3/12) ...
Generating image for latent code (4/12) ...
Generating image for latent code (5/12) ...
Generating image for latent code (6/12) ...
Generating image for latent code (7/12) ...
Generating image for latent code (8/12) ...
Generating image for latent code (9/12) ...
Generating image for latent code (10/12) ...
Generating image for latent code (11/12) ...
Generating image grid...
Generating latent vectors...
Generating interpolation video of length: 3.0 seconds...
[MoviePy] >>>> Building video out/lerp-1xslowdown.mp4
[MoviePy] Writing video out/lerp-1xslowdown.mp4


 99%|█████████▉| 90/91 [00:07<00:00, 12.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: out/lerp-1xslowdown.mp4 



4.0 Eigene Daten in den Latentspace projizieren

In [ ]:
rubens_w = project_image_into_wspace(Gs, 'rubens', save_video=True)
rembrandt_w = project_image_into_wspace(Gs, 'rembrandt')
cot_w = project_image_into_wspace(Gs, 'cot')
duerer_w = project_image_into_wspace(Gs, 'duerer', save_video=True)

duerer.jpg
Projector: Computing W midpoint and stddev using 10000 samples...
Projector: std = 35.7994
Projector: Setting up noise inputs...
Projector: Building image output graph...
Projector: Building loss graph...
Projector: Building noise regularization graph...
Projector: Setting up optimizer...
Projector: Preparing target images...
Projector: Initializing optimization state...


100%|██████████| 1000/1000 [03:20<00:00,  5.00it/s, dist=0.1820, loss=0.18]


In [ ]:
## FFHQ Model laden
tflib.init_tf()
with dnnlib.util.open_url("ffhq.pkl") as fp:
    _G, _D, Gs_ffhq = pickle.load(fp)

In [ ]:
## Duerer in den Latentspace von FFHQ projizieren
_ = project_image_into_wspace(Gs, 'duerer', save_video=True)

In [ ]:
# Interpolation zw. Duerer-Projektion und einem zufälligen Bild

duerer_w = np.load(outdir+'/duerer_dlatents.npz')['dlatents']
duerer_w.shape
w_za = Gs.components.mapping.run(np.random.normal(0,1,(1,512)), None)
w_duerer_za = interpolate(w_za, duerer_w, n_steps=12)
render_from_zs(Gs, w_duerer_za, w_space=True, grid=True, class_idx=None)

lerp_video(Gs,
               w_za,
               duerer_w,
               w_space=True,
               truncation_psi=1.0,         # Truncation trick
               slowdown=1,                 # Slowdown of the video (power of 2)
               duration_sec=4.0,          # Duration of video in seconds
               smoothing_sec=1.0,
               mp4_fps=30,
               mp4_codec="libx264",
               mp4_bitrate="16M")

Generating image for latent code (0/12) ...
Generating image for latent code (1/12) ...
Generating image for latent code (2/12) ...
Generating image for latent code (3/12) ...
Generating image for latent code (4/12) ...
Generating image for latent code (5/12) ...
Generating image for latent code (6/12) ...
Generating image for latent code (7/12) ...
Generating image for latent code (8/12) ...
Generating image for latent code (9/12) ...
Generating image for latent code (10/12) ...
Generating image for latent code (11/12) ...
Generating image grid...
Generating latent vectors...
(120, 1, 18, 512)
Generating interpolation video of length: 4.0 seconds...
[MoviePy] >>>> Building video out/lerp-1xslowdown.mp4
[MoviePy] Writing video out/lerp-1xslowdown.mp4


 99%|█████████▉| 120/121 [00:16<00:00,  7.31it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: out/lerp-1xslowdown.mp4 



In [ ]:
# Interpolation zwischen 3 Bildern
zs = [np.random.normal(0,1,(1,512)) for i in range(3)]
zs.append(np.mean(zs, axis=0))
render_from_zs(Gs, zs, grid=True)

Generating image for latent code (0/4) ...
Generating image for latent code (1/4) ...
Generating image for latent code (2/4) ...
Generating image for latent code (3/4) ...
Generating image grid...


5.0 Skalierung im W-Space

In [ ]:
zs = np.random.normal(0,1,(6,512))
zs_w = Gs.components.mapping.run(zs, None)
zs_w_b = zs_w * 8
zs_w *= 0

lerp_video(Gs,
               zs_w,
               zs_w_b,
               w_space=True,
               truncation_psi=1.0,         # Truncation trick
               slowdown=1,                 # Slowdown of the video (power of 2)
               duration_sec=5.0,          # Duration of video in seconds
               smoothing_sec=1.0,
               mp4_fps=30,
               mp4_codec="libx264",
               mp4_bitrate="16M")

1.0
Generating image for latent code (0/6) ...
Generating image for latent code (1/6) ...
Generating image for latent code (2/6) ...
Generating image for latent code (3/6) ...
Generating image for latent code (4/6) ...
Generating image for latent code (5/6) ...
Generating image grid...
2.0
Generating image for latent code (0/6) ...
Generating image for latent code (1/6) ...
Generating image for latent code (2/6) ...
Generating image for latent code (3/6) ...
Generating image for latent code (4/6) ...
Generating image for latent code (5/6) ...
Generating image grid...
4.0
Generating image for latent code (0/6) ...
Generating image for latent code (1/6) ...
Generating image for latent code (2/6) ...
Generating image for latent code (3/6) ...
Generating image for latent code (4/6) ...
Generating image for latent code (5/6) ...
Generating image grid...
7.999999999999999
Generating image for latent code (0/6) ...
Generating image for latent code (1/6) ...
Generating image for latent code (2